In [4]:
!pip install names
!pip install faker

DEPRECATION: Loading egg at c:\users\asus\anaconda3\lib\site-packages\holehe-1.61-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 21.7 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\asus\anaconda3\lib\site-packages\holehe-1.61-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [12]:
import sqlite3
import random
from datetime import date, timedelta

# Create database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Create tables
with open('schema.sql', 'r') as f:
    schema_sql = f.read()
cursor.executescript(schema_sql)


# Insert product categories
categories = [
    ('Electronics', 'Phones, laptops, tablets'),
    ('Clothing', 'Shirts, pants, dresses'),
    ('Books', 'Fiction and non-fiction'),
    ('Home', 'Furniture and decor'),
    ('Sports', 'Equipment and apparel'),
    ('Beauty', 'Skincare and makeup')
]
cursor.executemany('INSERT INTO product_categories (category_name, description) VALUES (?, ?)', categories)

# Generate 200 customers
first_names = ['John', 'Jane', 'Mike', 'Sarah', 'David', 'Lisa', 'Chris', 'Emily']
last_names = ['Smith', 'Johnson', 'Brown', 'Davis', 'Wilson', 'Miller', 'Taylor']
tiers = ['Bronze', 'Silver', 'Gold', 'Platinum']

customers = []
for i in range(200):
    first = random.choice(first_names)
    last = random.choice(last_names)
    customers.append((
        first,
        last,
        f"{first.lower()}.{last.lower()}{i}@email.com",
        f"555-{random.randint(100,999)}-{random.randint(1000,9999)}",
        f"2023-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
        random.choice(tiers),
        random.choice([True, False])
    ))

cursor.executemany('''
    INSERT INTO customers (first_name, last_name, email, phone, registration_date, customer_tier, is_active)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', customers)

# Generate 100 products
products = []
for i in range(100):
    cost = round(random.uniform(5, 200), 2)
    price = round(cost * random.uniform(1.2, 2.5), 2)  # 20-150% markup
    products.append((
        f"Product {i+1}",
        random.randint(1, 6),  # category_id
        price,
        cost,
        random.randint(0, 500),
        round(random.uniform(0.1, 10), 3),
        round(random.uniform(3.0, 5.0), 2),
        random.choice([True, False]),
        f"2023-{random.randint(1,12):02d}-{random.randint(1,28):02d}"
    ))

cursor.executemany('''
    INSERT INTO products (product_name, category_id, price, cost, stock_quantity, weight_kg, rating, is_available, created_date)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', products)

# Generate 1000+ orders
orders = []
addresses = ['123 Main St', '456 Oak Ave', '789 Pine Rd', '321 Elm St', '654 Maple Dr']
payment_methods = ['Credit Card', 'Debit Card', 'PayPal', 'Bank Transfer']
statuses = ['Pending', 'Confirmed', 'Shipped', 'Delivered', 'Cancelled']

for i in range(1000):
    orders.append((
        random.randint(1, 200),  # customer_id
        f"2024-{random.randint(1,6):02d}-{random.randint(1,28):02d}",  # order_date
        random.choice(statuses),
        round(random.uniform(25, 500), 2),  # total_amount
        random.choice(addresses),
        random.choice(payment_methods)
    ))

cursor.executemany('''
    INSERT INTO orders (customer_id, order_date, order_status, total_amount, shipping_address, payment_method)
    VALUES (?, ?, ?, ?, ?, ?)
''', orders)

# Generate order items (2-5 items per order)
order_items = []
for order_id in range(1, 1001):
    num_items = random.randint(2, 5)
    used_products = set()
    
    for _ in range(num_items):
        product_id = random.randint(1, 100)
        # Ensure unique product per order (as per UNIQUE constraint)
        while product_id in used_products:
            product_id = random.randint(1, 100)
        used_products.add(product_id)
        
        # Get product price
        cursor.execute('SELECT price FROM products WHERE product_id = ?', (product_id,))
        result = cursor.fetchone()
        unit_price = result[0] if result else round(random.uniform(10, 100), 2)
        
        order_items.append((
            order_id,
            product_id,
            random.randint(1, 3),  # quantity
            unit_price
        ))

cursor.executemany('''
    INSERT INTO order_items (order_id, product_id, quantity, unit_price)
    VALUES (?, ?, ?, ?)
''', order_items)

# Commit and close
conn.commit()
conn.close()

print("Database created successfully!")
print("Tables populated:")
print("   - product_categories: 6 rows")
print("   - customers: 200 rows") 
print("   - products: 100 rows")
print("   - orders: 1000 rows")
print("   - order_items: 3000+ rows")

✅ Database created successfully!
📊 Tables populated:
   - product_categories: 6 rows
   - customers: 200 rows
   - products: 100 rows
   - orders: 1000 rows
   - order_items: 3000+ rows
